In [50]:
import argparse
import os
import sys
import random

import pickle
import torch
from pprint import pprint

import common.cli_logger as cli_logger
import evaluation
from common.registerable import Registrable
from common.utils import dump_cfg, init_cfg
from components.dataset import Dataset
import model.parser

# Evaluators
import datasets.conala.conala_evaluator as conala_evaluator
import datasets.django.django_evaluator as django_evaluator

## CoNaLa

In [74]:
file_path = './experiments/conala.test.decode'
test_data = Dataset.from_bin_file('../tranx-data/data/conala/test_2.var_str_sep.bin')

In [75]:
decoded = pickle.load(open(file_path, 'rb'))
assert len(test_data) == len(decoded)

In [186]:
i = random.randint(0, len(decoded))

print(i)
print("NL:", " ".join(test_data.examples[i].src_sent))
print("target code:", test_data.examples[i].tgt_code)

[x.code for x in decoded[i]]

264
NL: find 10 largest differences between each respective elements of list var_0 and list var_1
target code: heapq.nlargest(10, range(len(var_0)), key=lambda i: abs(var_0[i] - var_1[i]))


['[(a + b) for a, b in zip(var_0, var_1)]',
 '[max(x) for x in var_0]',
 '[map(int, var_0) for x in var_1]',
 '[(a + y) for a, y in zip(var_0, var_1)]',
 '[max(x) for x in var_1]',
 '[(a + b) for a, y in zip(var_0, var_1)]',
 '[item for x in var_0 if x[1] == 10]',
 '[(a + y) for a, b in zip(var_0, var_1)]',
 '[max(map(int, x) for x in var_1) for x in var_1]',
 '[dict(map(int, x) for x in var_1) for x in var_1]',
 '[sum(map(int, x) for x in var_1) for x in var_1]',
 '[dict(map(int, int) for x in var_1) for x in var_1]',
 '[x for y, x in enumerate(var_0) if x in var_1]',
 '[dict(map(int, x) for x in var_1) for i in var_1]']

In [214]:
correct = []
for i in range(len(decoded)):
    correct.extend([x for x in decoded[i] if x.is_correct])

## Django

In [191]:
file_path = './experiments/django.test.decode'
test_data = Dataset.from_bin_file('../tranx-data/data/django/test_2.bin')

In [192]:
decoded = pickle.load(open(file_path, 'rb'))
assert len(test_data) == len(decoded)

In [216]:
i = random.randint(0, len(decoded))

print(i)
print(" ".join(test_data.examples[i].src_sent), "\n")
print(test_data.examples[i].tgt_code)

[x.code for x in decoded[i]]

1446
define the function blankout with 2 arguments : src and char . 

def blankout(src, char):
    pass


['def blankout(src, char):\n    pass',
 'def blankout(src, char, *args):\n    pass',
 'def blankout(src, char, **kwargs):\n    pass',
 'def blankout(src, char, *args, **kwargs):\n    pass',
 'def blankout(src, char=None, *args, **kwargs):\n    pass',
 'def blankout(src, char, *char):\n    pass',
 'def blankout(src, char, *char, **kwargs):\n    pass',
 'def blankout(src, char, **args):\n    pass',
 'def blankout(src, char=[], *args, **kwargs):\n    pass',
 'def blankout(src, char=(), *args, **kwargs):\n    pass',
 'def blankout(src, char, **blankout):\n    pass',
 'def blankout(src, char, **options):\n    pass',
 'def blankout(src, char=1, *args, **kwargs):\n    pass',
 "def blankout(src, char='', *args, **kwargs):\n    pass",
 'def blankout(src, char=blankout(src), *args, **kwargs):\n    pass']

In [217]:
parser_cls = Registrable.by_name('default_parser')
parser = parser_cls.load(model_path='../tranx-data/data/pretrained_models/django_2.bin', cuda=False)
parser.eval()

sent = "substitute 1 for x".split()
sent = test_data.examples[random.randint(0, len(test_data))].src_sent
hyps = parser.parse(sent, context=None, beam_size=15)

assert len(hyps) > 0

hyps

for i, hyp in enumerate(hyps):
    got_code = False

    try:
        hyp.code = model.transition_system.ast_to_surface_code(hyp.tree)
        print("ok >", hyp.code, "\n")
    except Exception as e:
        print(e)

Getting registered default_parser
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
module 'model' has no attribute 'transition_system'
